# Probostat Data Processing Notebook
This notebook provides a concise and user-friendly interface to work with Probostat data files. Follow the steps to load, view, manipulate, and export your data.


---

## Step 0: Installing external packages missing on the system
This project is using Python packages developed by members of the open-source community, and they are necessary for you to be able to run these scripts.  
If you have a fresh install of Python on this machine, you will need to install them.  

If you already installed the prerequisites, feel free to skip.

In [ ]:
#!pip install -r requirements.txt


## Step 1: Load Probostat Files
First, we'll load the Probostat data files. Please make sure the files are in CSV format.

In [3]:
import brlopack

### Initialize the BrloPack object

In [ ]:
brlo_pack = brlopack.brlopack()

In [ ]:
# List of Probostat files to load
probostat_files = [
    'data/probostat/new/BCTZ_16cikel za lukata loceno-2 point resistances.csv',
    'data/probostat/new/BCTZ_16cikel za lukata loceno-Flow.csv',
    'data/probostat/new/BCTZ_16cikel za lukata loceno-Keysiht MC.csv',
    'data/probostat/new/BCTZ_16cikel za lukata loceno-Temp.csv'
]

### Load the files into the BrloPack object
**Note**: Loading Probostat can take up to 1 or 2 minutes, because it is running a costly stitching operation over these `.csv` tables. Please be patient

In [ ]:
brlo_pack.tellFiles(probostat_files)
brlo_pack.loadFiles()


## Step 2: View Data Structure and Content
You can explore the files, tables, and columns within the loaded data.


In [ ]:
# Example: View the files loaded
loadedFiles = brlo_pack.tellMeFiles()
loadedFiles

**Question 1: Why does this show up as one file** (named 'Probostat') **; when we loaded 4?**  
*Answer 1*
<details><summary>Click here</summary>

In the case of Probostat it is assumed that multiple loaded `.csv` files should be merged/stitched into a single `DataFrame`/table and it will show up as only one file (for now the default name is just `Probostat`).  

*Why?* Usually with Probostat multiple of those files refer to a single experiment/measurement period.
</details>

---

**Question 2: Ok, that's why it is only one file. But what if I want multiple Probostat files to stay separate files?**  
*Answer 2*
<details><summary>Click here</summary>

Shortly - `.csv` files WILL be merged, `.xlsx` files WILL NOT be merged.  
The `.xslx` files can be obtained by using the method `[brlopack object name here].exportToExcel()`. 

**Specific example of usage**
<details><summary>Click here</summary>

If you want to work with multiple files, each representing one experiment - it's not a problem. After you stitch several `.csv` files into a single `DataFrame`/table, use `paket.exportToExcel()` to get a single `.xslx` file for that experiment.  

*Then*, do that for each experiment (i.e. several related `.csv` files). Then load those `.xlsx` files together - and they will stay separate! 
</details>
</details>

---

**Question 3 (not important, feel free to skip): Why does it work like this?**  
*Answer 3*
<details><summary>Click here</summary>

It made sense for the specific constraints that we had at the time. Origin had a lot of problems when exporting all of the measured data from multiple sensors into a single `.csv` file. We had to export data of 2-3 sensors at the most per `.csv` file. Sensors did not necessarily all start to measure from the same point in time, nor did they finish at the same point in time, leading to a lot of `NaN` ("Not a number") values in the table and to crashing. That's why we stitch them back here in the `brlopack` Python package (and for the usecase thus far, we could always assume that Probostat's `.csv` files always need to be stitched)

</details>

---

#### Now let us move onto seeing the table within that file!

In [ ]:
# Example: View the tables in a specific file
tableNames = brlo_pack.tellMeTablesInFile(loadedFiles[0])
tableNames

#### Finally this is how we can see all of the columns in this table!

In [ ]:
# Example: View the columns in a specific table
columnNames = brlo_pack.tellMeColumnsInTable(loadedFiles[0], tableNames[0])
columnNames

## Step 3: Manipulate Data
Here you can perform essential data manipulations, such as unit conversion.


Here is a list of all of the operations that you can do (methods defined in the `brlopack` class):
 1. `divideConstant(columnName, newColumnName, constName)`
 2. `multiplyConstant(columnName, newColumnName, constName)`
 3. `subtractConstant(columnName, newColumnName, constName)`
 4. `addConstant(columnName, newColumnName, constName)`
 5. `inverseColumn(columnName, newColumnName, constName=None)`
 6. `squareColumn(columnName, newColumnName, constName=None)`
 7. `sqrtColumn(columnName, newColumnName, constName=None)`
 8. `averageTwoColumns(columnName, newColumnName, secondColumn)`
 9. `changeUnitOfConstant(constantName, unitPrefix)`
 10. `logN(columnName, newColumnName, constName=None)`
 11. `logConstant(columnName, newColumnName, constName)`


In [ ]:
# Just put in any of the operations from above
brlo_pack.inverseColumn(columnName=columnNames[1], "Resistance inverted [1/ohm]")

**Warning**: Make sure that you write the proper unit of measurement in SQUARE BRACKETS.
Otherwise unit conversions will not work, and the code might even crash.

---

**For advanced users - how to do any operation** beyond just these
<details><summary>Click here</summary>

You can open the `brlopack.py` to see how these ones were defined. You will notice they all follow the same pattern of arguments:
1. `columnName` (str) - the input column
2. `newColumnName` (str) - the output column
3. `constName` (str, optional) - the constant used somehow in the operation

You can also make your own method in this style using the `doOperation(operation, columnName, newColumnName, constName)` method.  

Just peek into the `brlopack.py` file to see how it's done. It utilizes the so-called "lambda functions" or alternatively you can assign a function to a variable (yes, this is possible in Python - you can store functions in variables and pass them to other functions).

If you are unsure about anything - do not hesitate to contact me over email or WhatsApp with your questions (especially if you tried to Google them or ask ChatGPT about it beforehand). If you don't have my contact info, ask around the office, someone has it!
</details>


## Step 4: Export Processed Data
Finally, you can export the processed data to an Excel file.

In [ ]:
# Export the data to an Excel file
brlo_pack.exportToExcel()

## Conclusion
You have successfully loaded, viewed, manipulated, and exported Probostat data. Feel free to modify the code cells to work with your specific data and requirements! You may also contact me with any questions you might have over email or WhatsApp (ask around the office).